In [ ]:
import os
import sys
sys.path.insert(0,"..")
import numpy as np
import matplotlib.pyplot as plt
import pyemu
print(pyemu.__file__)
import flopy
import platform
from pathlib import Path
import shutil
import pandas as pd

In [ ]:
t_d = "template"
if not os.path.exists(t_d):
    raise Exception("need to run simple_pstfrom notebook")

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"at.pst"))
pst.npar,pst.nobs

In [ ]:
pst.pestpp_options

In [ ]:
if "ies_par_en" not in pst.pestpp_options:
    raise Exception("prior par ensemble not found")

In [ ]:
pst.pestpp_options = {"ies_par_en":pst.pestpp_options["ies_par_en"]}
pst.pestpp_options["ies_num_reals"] = 50
pst.pestpp_options["ies_bad_phi_sigma"] = 1.75
pst.pestpp_options

In [ ]:
pst.control_data.noptmax = 4

In [ ]:
pst.obs_groups

In [ ]:
weighted_groups = [i for i in pst.obs_groups if i.endswith('group')]
weighted_groups

In [ ]:
wts = [.4,.3,.2,.2]

In [ ]:
with open(f'{t_d}/phi_factors.csv', 'w') as ofp:
    [ofp.write(f'{g},{w}\n') for g,w in zip(weighted_groups,wts)]

In [ ]:
pst.pestpp_options['ies_phi_factor_file']='phi_factors.csv'

In [ ]:
pst.obs_groups

In [ ]:
weighted_groups = [i for i in pst.obs_groups if i.endswith('group')]
weighted_groups

In [ ]:
wts = [.4,.3,.2,.2]

In [ ]:
with open(f'{t_d}/phi_factors.csv', 'w') as ofp:
    [ofp.write(f'{g},{w}\n') for g,w in zip(weighted_groups,wts)]

In [ ]:
pst.pestpp_options['ies_phi_factor_file']='phi_factors.csv'

In [ ]:
pst.write(os.path.join(t_d,"at.pst"),version=2)

In [ ]:
m_d = "master_ies_simple"

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-ies","at.pst",num_workers=15,master_dir=m_d,worker_root='.')

In [ ]:
def plot_ies_results(m_d,tag=""):
    pst = pyemu.Pst(os.path.join(m_d,"at.pst"))
    
    noise = pst.ies.noise
    itrs = pst.ies.phiactual.iteration.values
    obs = pst.observation_data
    nzobs = obs.loc[obs.weight > 0,:]
    odict = {n:v for n,v in zip(nzobs.obsnme,nzobs.obsval)}
    ptoe = pst.ies.get("obsen{0}".format(itrs.max()))
    proe = pst.ies.obsen0
    phivals = pst.ies.phimeas.iloc[-1,6:]
    phimean = phivals.mean()
    phistd = phivals.std()
    thresh = phimean + (phistd*1.75)
    keep = phivals[phivals<=thresh].index.values
    ptoe = ptoe.loc[keep]
    proe = proe.loc[keep]
    fig,ax = plt.subplots(1,1,figsize=(6,6))
    names = obs.loc[obs.obgnme=="headgroup","obsnme"].values
    for oname in names:
        if "at.csv" not in oname:
            continue
        ovals = [odict[oname] for _ in range(noise.shape[0])]
        ax.scatter(ovals,noise[oname],marker='.',c='r',s=100,alpha=0.25)
        
        ovals = [odict[oname] for _ in range(ptoe.shape[0])]
        ax.scatter(ovals,ptoe[oname],marker='.',c='b',alpha=0.5)
        #ylim = ax.get_ylim()
        ovals = [odict[oname] for _ in range(proe.shape[0])]
        ax.scatter(ovals,proe[oname],marker='.',c='0.5',alpha=0.5,zorder=0)
        #ax.set_ylim(ylim)
        #ax.set_xlim(ylim)
    mn,mx = noise.loc[:,names].values.min(),noise.loc[:,names].values.max()
    ax.plot([mn,mx],[mn,mx],"k--",lw=3)
    ax.set_xlabel("obs")
    ax.set_ylabel("sim")
    ax.set_xlim(mn,mx)
    ax.set_title(tag)
    
    _ = ax.set_ylim(mn,mx)

    kobs = obs.loc[obs.obsnme.str.contains("k_aq"),:].copy()
    kobs['i'] = kobs.i.astype(int)
    kobs['j'] = kobs.j.astype(int)

    prarr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
    prarr[kobs.i,kobs.j] = np.log10(ptoe.loc[:,kobs.obsnme].mean())
    ptarr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
    ptarr[kobs.i,kobs.j] = np.log10(ptoe.loc["base",kobs.obsnme])
    fig,axes = plt.subplots(1,2,figsize=(8,4))
    cb = axes[0].imshow(prarr,vmin=ptarr.min())
    plt.colorbar(cb,ax=axes[0])
    axes[0].set_title("mean "+tag)
    
    cb = axes[1].imshow(ptarr,vmax=prarr.max())
    plt.colorbar(cb,ax=axes[1])
    axes[1].set_title("min err var "+tag)

    prarr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
    prarr[kobs.i,kobs.j] = np.log10(proe.loc[:,kobs.obsnme].std())
    ptarr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
    ptarr[kobs.i,kobs.j] = np.log10(ptoe.loc[:,kobs.obsnme].std())
    fig,axes = plt.subplots(1,2,figsize=(8,4))
    cb = axes[0].imshow(prarr,vmin=ptarr.min())
    plt.colorbar(cb,ax=axes[0])
    axes[0].set_title("prior stdev "+tag)
    cb = axes[1].imshow(ptarr,vmax=prarr.max())
    plt.colorbar(cb,ax=axes[1])
    axes[1].set_title("posterior stdev "+tag)

    bobs = obs.loc[obs.oname=="at.budget.csv",:]
    for oname in bobs.obsnme:
        if "chd" not in oname:
            continue
        fig,ax = plt.subplots(1,1,figsize=(6,6))
        ax.hist(proe.loc[:,oname],bins=10,fc="0.5",alpha=0.5,density=True)
        ax.hist(ptoe.loc[:,oname],bins=10,fc="b",alpha=0.5,density=True)
        ax.set_title(tag+" "+oname)
        ax.set_yticks([])
    
    names = obs.loc[obs.obgnme=="headgroup","obsnme"].values
    noise = pst.ies.noise
    proe = pst.ies.obsen0
    ptoe = pst.ies.get("obsen{0}".format(pst.ies.phiactual.iteration.max()))
    phivals = pst.ies.phimeas.iloc[-1,6:]
    phimean = phivals.mean()
    phistd = phivals.std()
    thresh = phimean + (phistd*1.75)
    keep = phivals[phivals<=thresh].index.values
    fig,axes = plt.subplots(2,1,figsize=(10,20))
    actual,meas = pst.ies.phiactual.iloc[:,6:],pst.ies.phimeas.iloc[:,6:]
    actual = actual.loc[:,keep]
    meas = meas.loc[:,keep]
    
    axes[0].hist(np.log10(actual.iloc[0,:].values),fc="0.5",alpha=0.5)
    axes[0].hist(np.log10(actual.iloc[-1,:].values),fc="b",alpha=0.5)
    axes[0].set_title('log-10 phi w/o noise')

    axes[1].hist(np.log10(meas.iloc[0,:].values),fc="0.5",alpha=0.5)
    axes[1].hist(np.log10(meas.iloc[-1,:].values),fc="b",alpha=0.5)
    axes[1].set_title('log-10 phi w/ noise')
    
    
    fig,axes = plt.subplots(2,1,figsize=(10,20))
    ptoe = ptoe.loc[keep,:]
    nzobs.sort_values(by="obsval",inplace=True)
    ovals = nzobs.obsval.values
    for real in ptoe.index:
        nvals = noise.loc[real,nzobs.obsnme].values   
        prvals = proe.loc[real,nzobs.obsnme].values
        ptvals = ptoe.loc[real,nzobs.obsnme].values
        ax = axes[0]
        #ax.plot(ovals,nvals,"r-",alpha=0.5,lw=0.5)
        ax.scatter(ovals,nvals,marker='.',alpha=0.2,s=70,c='r')
        #ax.plot(ovals,prvals,"0.5",alpha=0.2,lw=0.3,dashes=(1,1))
        #ax.plot(ovals,ptvals,"b",alpha=0.5,lw=0.5)#,marker='.',ms=5)
        ax.scatter(ovals,ptvals,marker='.',s=50,c='b')
        ax = axes[1]
        ax.plot(ovals,nvals,"r-",alpha=0.5,lw=0.5,zorder=0)
        ax.scatter(ovals,nvals,marker='.',alpha=0.2,s=70,c='r')
        #ax.plot(ovals,prvals,"0.5",alpha=0.2,lw=0.3,dashes=(1,1))
        #ax.plot(ovals,ptvals,"b",alpha=0.5,lw=0.5,zorder=0)#,marker='.',ms=5)
        ax.scatter(ovals,ptvals,marker='.',s=50,c='b')
    
    mn,mx = noise.loc[:,names].values.min(),noise.loc[:,names].values.max()
    for ax in axes:
        ax.plot([mn,mx],[mn,mx],"k--",lw=3)
        ax.set_xlabel("observed")
        ax.set_ylabel("simulated")
        ax.set_xlim(mn,mx)
        ax.grid()
    
    _ = ax.set_ylim(mn,mx)
            
    


In [ ]:
plot_ies_results(m_d,tag="uncor noise")

In [ ]:
pst.pestpp_options["ies_no_noise"] = True
pst.write(os.path.join(t_d,"at.pst"),version=2)
m_d = "master_ies_nonoise"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","at.pst",num_workers=15,master_dir=m_d,worker_root='.')
plot_ies_results(m_d,"no noise")

In [ ]:
#noise_vals = np.linspace(-2,2,pst.ies.paren0.shape[0])
noise_vals = np.random.normal(0.0,1.5,pst.ies.paren0.shape[0])
noise_vals

In [ ]:
obs = pst.observation_data
nzobs = obs.loc[obs.weight>0,:]

In [ ]:

opn = []
for noise_val in noise_vals:
    opn.append(nzobs.obsval+noise_val)
df= pd.DataFrame(opn,index=np.arange(len(opn),dtype="int"))

df.to_csv(os.path.join(t_d,"noise_steps.csv"))
pst.pestpp_options["ies_obs_en"] = "noise_steps.csv"
pst.pestpp_options.pop("ies_no_noise",None)

In [ ]:
pst.write(os.path.join(t_d,"at.pst"),version=2)
m_d = "master_ies_corrnoise"
pyemu.os_utils.start_workers(t_d,"pestpp-ies","at.pst",num_workers=15,master_dir=m_d,worker_root='.')
plot_ies_results(m_d,"corrnoise")